In [26]:
import numpy as np
import tensorflow as tf
from utils.draw import draw_single
print(f'Tensorflow v{tf.__version__}')

Tensorflow v2.0.0


In [27]:
def generate(count):
    X = np.random.randint(0, high=256, size=(count, 9))
    a = np.array([[1, 1, 1, 0, 0, 0, 0, 0, 0], 
                  [0, 0, 0, 1, 1, 1, 0, 0, 0], 
                  [0, 0, 0, 0, 0, 0, 1, 1, 1]])

    Y = np.argmax(X.dot(a.T), axis=1)
    return X, Y

In [28]:
batch = 500
epochs = 10

def convert(x, y): return tf.dtypes.cast(x, tf.float32)/255., tf.one_hot(y, depth=3)

train_ds = tf.data.Dataset.from_tensor_slices(generate(60000))
train_ds = train_ds.map(convert)
train_ds = train_ds.shuffle(buffer_size=1000)
train_ds = train_ds.batch(batch)
train_ds = train_ds.repeat(epochs)

# Setting things up!

In [29]:
class MySquareModel(tf.keras.layers.Layer):

    def __init__(self):
        super(MySquareModel, self).__init__()
        self.w = self.add_weight(shape=(9, 3),
                                initializer='random_uniform',
                                trainable=True)
        self.b = self.add_weight(shape=(3,),
                                initializer='zeros',
                                trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [38]:
model = tf.keras.models.Sequential([MySquareModel()])
img = tf.constant([np.array([123, 245, 145, 12, 134, 22, 255, 43, 231]) / 255.], dtype='float32')
model(img)

<tf.Tensor: id=362185, shape=(1, 3), dtype=float32, numpy=array([[-0.02624908, -0.15207854,  0.0255867 ]], dtype=float32)>

In [39]:
model.trainable_variables

[<tf.Variable 'Variable:0' shape=(9, 3) dtype=float32, numpy=
 array([[-0.01373247, -0.01200256,  0.03546797],
        [-0.02912207, -0.04413385,  0.03940875],
        [-0.04820472, -0.00708705,  0.04478318],
        [ 0.01264418,  0.0268553 ,  0.04705197],
        [ 0.03455373, -0.02295297, -0.01336062],
        [-0.00938047, -0.04129829, -0.01868693],
        [-0.01318527, -0.04221733, -0.0073367 ],
        [ 0.02679801, -0.01586165, -0.01295799],
        [ 0.02924043, -0.04482177, -0.04295149]], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

# The Optimization Loop

In [30]:
# set up all teh things
cost = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.SGD(learning_rate=.01)

model = tf.keras.models.Sequential([MySquareModel()])

train_loss = tf.keras.metrics.Mean()
train_accuracy = tf.keras.metrics.Accuracy()

In [31]:
i = 0
for X, Y in train_ds:
    with tf.GradientTape() as tape:
        pred = model(X)
        loss = cost(pred, Y)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss(loss)
    train_accuracy(tf.math.argmax(Y, axis=1), tf.math.argmax(logits, axis=1))
    i+=1
    if i % 50 == 0:
        print(f'Loss: {train_loss.result()}, Accuracy: {train_accuracy.result()}')

Loss: 0.26732805371284485, Accuracy: 0.5679200291633606
Loss: 0.24045701324939728, Accuracy: 0.6398199796676636
Loss: 0.22788876295089722, Accuracy: 0.6955066919326782
Loss: 0.22011196613311768, Accuracy: 0.7339400053024292
Loss: 0.2144068032503128, Accuracy: 0.762503981590271
Loss: 0.2097979635000229, Accuracy: 0.7844866514205933
Loss: 0.20583276450634003, Accuracy: 0.8025485873222351
Loss: 0.20230117440223694, Accuracy: 0.8171949982643127
Loss: 0.19912317395210266, Accuracy: 0.8295644521713257
Loss: 0.19616088271141052, Accuracy: 0.8403319716453552
Loss: 0.1934303641319275, Accuracy: 0.849523663520813
Loss: 0.19085697829723358, Accuracy: 0.8577399849891663
Loss: 0.18841469287872314, Accuracy: 0.8648861646652222
Loss: 0.18610899150371552, Accuracy: 0.8714942932128906
Loss: 0.18389859795570374, Accuracy: 0.8773066401481628
Loss: 0.18180766701698303, Accuracy: 0.8825349807739258
Loss: 0.17980016767978668, Accuracy: 0.8874306082725525
Loss: 0.17788781225681305, Accuracy: 0.89182668924331

# Trying it out (inference)

In [40]:
labels = ['top', 'middle', 'bottom']
img = tf.constant([np.array([123, 245, 145, 12, 134, 22, 255, 43, 231]) / 255.], dtype='float32')

In [41]:
Y = model(img)
answer = tf.math.argmax(Y, axis=1).numpy()[0]
labels[answer]

'bottom'